This file contains R functions that reproduce the analysis permormed in the discrete network from **Herrera et al. 2024. A system-Level Model Reveals that Transcriptional Stochasticity is Required for Hematopoietic Stem Cell Differentiation**

**1. Attractors identification**

In [9]:
# Load the BoolNet package, installing it if necessary
if (!require('BoolNet')) {
  install.packages('BoolNet')
}
library('BoolNet')

# Load the necessary packages for PDF generation
if (!require('gridExtra')) {
  install.packages('gridExtra')
}
library('gridExtra')

if (!require('ggplot2')) {
  install.packages('ggplot2')
}
library('ggplot2')

if (!require('grid')) {
  install.packages('grid')
}
library('grid')

if (!require('dplyr')) {
  install.packages('dplyr')
}
library('dplyr')

# Load a Boolean network from a file
network <- loadNetwork('BoolNet_HSC_network.txt')

# Find the attractors of the network using synchronous update
synchronous_attractors <- getAttractors(network, type = "synchronous")

# Find the attractors of the network using asynchronous update
#asynchronous_attractors <- getAttractors(network, type = "asynchronous")

# Obtain the basin sizes of the synchronous attractors
synchronous_basin_sizes <- sapply(synchronous_attractors$attractors, function(attractor) attractor$basinSize)

# Calculate the total possible states of the network
total_states <- 2^length(network$genes)

# Create a data frame with the basin sizes of the synchronous attractors
synchronous_basin_sizes_df <- data.frame(
  Attractor = paste0("Synchronous_", seq_along(synchronous_basin_sizes)),
  BasinSize = synchronous_basin_sizes
)
synchronous_basin_sizes_df <- synchronous_basin_sizes_df %>%
  mutate(Cell_Identity = c("MEP", "MEP", "MEP", "MEP", "MEP", "MEP", "CLP", "CLP", "GMP", "GMP","GMP","GMP", "HSC","NS","NS","NS","NS","NS")) #NS = Not specified 

# Create a bar plot with the basin sizes of the synchronous attractors (including "NS")
synchronous_basin_sizes_df$Attractor <- factor(
  synchronous_basin_sizes_df$Attractor, 
  levels = synchronous_basin_sizes_df$Attractor
)
bar_plot <- ggplot(synchronous_basin_sizes_df, aes(x = Attractor, y = BasinSize, fill = Cell_Identity)) +
  geom_bar(stat = "identity") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
  labs(title = "Basin Sizes of Synchronous Attractors", x = "Attractor", y = "Basin Size")

# Create a table with the attractor number, its basin size, and the percentage (including "NS")
synchronous_basin_table <- data.frame(
  Attractor = seq_along(synchronous_basin_sizes_df$BasinSize),
  Initial_States = synchronous_basin_sizes_df$BasinSize,
  Relative_Size = (synchronous_basin_sizes_df$BasinSize / total_states) * 100,
  Cell_Identity = synchronous_basin_sizes_df$Cell_Identity
)

# Exclude attractors with Cell_Identity as "NS" for the summary table
filtered_basin_sizes_df <- synchronous_basin_sizes_df %>%
  filter(Cell_Identity != "NS")

# Recalculate total states for valid attractors (excluding "NS")
filtered_total_states <- sum(filtered_basin_sizes_df$BasinSize)

# Create a summary table with the recalculated percentage (excluding "NS")
filtered_basin_summary <- filtered_basin_sizes_df %>%
  group_by(Cell_Identity) %>%
  summarise(Total_BasinSize = sum(BasinSize), 
            Relative_Size = (sum(BasinSize) / filtered_total_states) * 100)

# Create a PDF document
pdf("Discrete_attractors_report.pdf", width = 8, height = 10)

# Add synchronous attractors
plotAttractors(synchronous_attractors, title = "Synchronous attractors", onColor = "black", offColor = "gray", borderColor = "white")

# Add bar plot of basin sizes (including "NS")
#print(bar_plot)

# Add table of basin sizes (including "NS")
grid.newpage()
grid.text("Basin Sizes of Synchronous Attractors", gp = gpar(fontsize = 20, fontface = "bold"), y = unit(0.9, "npc"))
grid.table(synchronous_basin_table, vp = viewport(y = unit(0.4, "npc")))

# Add table of summary (excluding "NS")
grid.newpage()
grid.text("Basin Sizes Summary (Excluding NS)", gp = gpar(fontsize = 20, fontface = "bold"), y = unit(0.9, "npc"))
grid.table(filtered_basin_summary, vp = viewport(y = unit(0.4, "npc")))

# Add asynchronous attractors
#plotAttractors(asynchronous_attractors, title = "Asynchronous attractors" , onColor = "black", offColor = "gray", borderColor = "white")

# Close the PDF document
dev.off()


**Synchronous mutation**

In [2]:
# Function to simulate mutation for each gene
simulate_mutation <- function(network, gene, mutation_type) {
  mutated_network <- fixGenes(network, gene, ifelse(mutation_type == "LOF", 0, 1))
  mutated_attractors <- getAttractors(mutated_network, type = "synchronous")
  
  # Obtain the basin sizes of the synchronous attractors
  mutated_basin_sizes <- sapply(mutated_attractors$attractors, function(attractor) attractor$basinSize)
  
  mutated_basin_sizes_df <- data.frame(
    Attractor = paste0("Mutated_", gene, "_", seq_along(mutated_basin_sizes)),
    BasinSize = mutated_basin_sizes
  )
  
  # Calculation of BasinSizePercentage
  mutated_basin_table <- data.frame(
    Attractor = seq_along(mutated_basin_sizes),
    Initial_States = mutated_basin_sizes,
    Relative_Size = (mutated_basin_sizes / sum(mutated_basin_sizes)) * 100
  )
  
  list(
    mutated_basin_sizes_df = mutated_basin_sizes_df,
    mutated_basin_table = mutated_basin_table,
    mutated_attractors = mutated_attractors
  )
}

# Function to create PDF and plot results
plot_mutation_results <- function(network, mutation_type) {
  pdf(paste0("synchronous_", mutation_type, "_attractors_and_basin_sizes.pdf"), width = 7, height = 10)
  
  for (gene in network$genes) {
    result <- simulate_mutation(network, gene, mutation_type)
    
    # Plot the attractors
    plotAttractors(
      result$mutated_attractors, 
      title = paste("Synchronous attractors for", mutation_type, "mutation in", gene), 
	  onColor = "black", 
	  offColor = "gray", 
	  borderColor = "white"
    )
    
    # Generate bar plot for basin sizes
    #barplot(
    #  result$mutated_basin_sizes_df$BasinSize,
    #  names.arg = result$mutated_basin_sizes_df$Attractor,
    #  las = 2,
    #  cex.names = 0.5, 
    #  col = "skyblue",
    #  main = paste("Basin Sizes for", mutation_type, "mutation in", gene),
    #  xlab = "Attractor",
    #  ylab = "Basin Size"
    #)
    
    # Add a new page for the table
    grid.newpage()
    
    # Display the table
    grid.text(
      paste("Basin sizes for", mutation_type, "mutation in", gene), 
      gp = gpar(fontsize = 20, fontface = "bold"), 
      y = unit(0.9, "npc")
    )
    grid.table(result$mutated_basin_table)
  }
  
  dev.off()
}

# Plot results for loss-of-function mutations
plot_mutation_results(network, "LOF")

# Plot results for gain-of-function mutations
plot_mutation_results(network, "GOF")

**Asynchronous mutations**

In [3]:
# Function to simulate mutation for each gene
simulate_mutation <- function(network, gene, mutation_type) {
  mutated_network <- fixGenes(network, gene, ifelse(mutation_type == "LOF", 0, 1))
  mutated_attractors <- getAttractors(mutated_network, type = "asynchronous")
  return(list(mutated_network = mutated_network, mutated_attractors = mutated_attractors))
}

# Function to create PDF and plot results
plot_mutation_results <- function(network, mutation_type) {
  pdf(paste0("asynchronous_", mutation_type, "_attractors.pdf"), width = 7, height = 10)
  
  for (gene in network$genes) {
    cat("Processing gene:", gene, "\n")  # Display a message for each iteration
    flush.console()  # Ensure the message is displayed immediately
    result <- simulate_mutation(network, gene, mutation_type)
    
    # Plot the attractors
    plotAttractors(
      result$mutated_attractors, 
      title = paste("Asynchronous attractors for", mutation_type, "mutation in", gene), 
      onColor = "black", 
      offColor = "gray", 
      borderColor = "white"
    )
  }
  
  dev.off()
}

# Plot results for loss-of-function mutations
plot_mutation_results(network, "LOF")

# Plot results for gain-of-function mutations
plot_mutation_results(network, "GOF")